In [ ]:
# basic library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# model
from catboost import CatBoostClassifier

# sampler
from imblearn.under_sampling import RandomUnderSampler

# metrics
from sklearn.metrics import roc_auc_score, f1_score

# custom modules
from utils import set_seed, make_submission_file
import preprocess.base as pp_base

### Global Setting

In [ ]:
hparams = {
    'seed': 42,
}

In [ ]:
set_seed(hparams['seed'])

### Data Preprocessing

In [ ]:
tr_data_path = "../Data/train.csv"
tt_data_path = "../Data/test.csv"

In [ ]:
tr_df, tt_df = pp_base.load_data(tr_data_path, tt_data_path)

In [ ]:
# "불임 원인 - 여성 요인", 제외
categorical_columns = [
        "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부",
        "배란 유도 유형", "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부",
        "착상 전 유전 진단 사용 여부", "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인",
        "여성 부 불임 원인", "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태", "배아 생성 주요 이유", "총 시술 횟수", "클리닉 내 총 시술 횟수",
        "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
        "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처", "난자 기증자 나이",
        "정자 기증자 나이", "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부",
        "대리모 여부", "PGD 시술 여부", "PGS 시술 여부"
        ]

numerical_columns = [
        "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수",
        "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", 
        "기증자 정자와 혼합된 난자 수", "난자 채취 경과일", "난자 해동 경과일",
        "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"
        ]

**`ID` 컬럼 제거**

In [ ]:
tr_df, tt_df = pp_base.drop_features(tr_df, tt_df, categorical_columns + numerical_columns)

**`특정 시술 유형` 컬럼의 결측치 제거 후 첫 번째 문자열을 기준으로 regroup**

In [ ]:
# 결측치 삭제 후 regroup
tr_df = tr_df.dropna(subset=['특정 시술 유형'])
tr_df['특정 시술 유형'] = tr_df['특정 시술 유형'].apply(lambda x: x.split()[0].split(":")[0])
tt_df['특정 시술 유형'] = tt_df['특정 시술 유형'].apply(lambda x: x.split()[0].split(":")[0])

**`배란 유도 유형` 컬럼에서 이상치 제거(세트로타이드, 생식선 ...)** >> 제외

In [ ]:
# tr_df = tr_df.drop(index=[
#     tr_df[tr_df['배란 유도 유형'] == '세트로타이드 (억제제)'].index.values[0],
#     tr_df[tr_df['배란 유도 유형'] == '생식선 자극 호르몬'].index.values[0]
#     ])

**`단일 배아 이식 여부` 컬럼에서 결측치 0.0으로 처리** >> 제외

In [ ]:
# tr_df['단일 배아 이식 여부'] = tr_df['단일 배아 이식 여부'].fillna(0.0)
# tt_df['단일 배아 이식 여부'] = tt_df['단일 배아 이식 여부'].fillna(0.0)

**`착상 전 유전 검사 사용 여부` 컬럼에서 결측치 0.0으로 처리**

In [ ]:
tr_df['착상 전 유전 검사 사용 여부'] = tr_df['착상 전 유전 검사 사용 여부'].fillna(0.0)
tt_df['착상 전 유전 검사 사용 여부'] = tt_df['착상 전 유전 검사 사용 여부'].fillna(0.0)

**`착상 전 유전 진단 사용 여부` 컬럼에서 결측치 0.0으로 처리** >> 제외

In [ ]:
# tr_df['착상 전 유전 진단 사용 여부'] = tr_df['착상 전 유전 진단 사용 여부'].fillna(0.0)
# tt_df['착상 전 유전 진단 사용 여부'] = tt_df['착상 전 유전 진단 사용 여부'].fillna(0.0)

**`배아 생성 주요 이유` 결측치를 최빈값으로 채운 뒤, 첫 문자열을 기준으로 regroup** >> 제외

In [ ]:
# tr_df['배아 생성 주요 이유'] = tr_df['배아 생성 주요 이유'].fillna('현재 시술용')
# tt_df['배아 생성 주요 이유'] = tt_df['배아 생성 주요 이유'].fillna('현재 시술용')

In [ ]:
# tr_df['배아 생성 주요 이유'] = tr_df['배아 생성 주요 이유'].apply(lambda x: x.split(",")[0].split(":")[0])
# tt_df['배아 생성 주요 이유'] = tt_df['배아 생성 주요 이유'].apply(lambda x: x.split(",")[0].split(":")[0])

**`동결 배아 사용 여부` 결측치 0.0으로 처리 후 사용** >> 제외

In [ ]:
# tr_df['동결 배아 사용 여부'] = tr_df['동결 배아 사용 여부'].fillna(0.0)
# tt_df['동결 배아 사용 여부'] = tt_df['동결 배아 사용 여부'].fillna(0.0)

**`신선 배아 사용 여부` 결측치 1.0으로 처리 후 사용** >> 제외

In [ ]:
# tr_df['신선 배아 사용 여부'] = tr_df['신선 배아 사용 여부'].fillna(1.0)
# tt_df['신선 배아 사용 여부'] = tt_df['신선 배아 사용 여부'].fillna(1.0)

**`기증 배아 사용 여부` 결측치 0.0으로 처리 후 사용** >> 보류

In [ ]:
# tr_df['기증 배아 사용 여부'] = tr_df['기증 배아 사용 여부'].fillna(0.0)
# tt_df['기증 배아 사용 여부'] = tt_df['기증 배아 사용 여부'].fillna(0.0)

**`대리모 여부` 결측치 0.0으로 처리 후 사용** >> 제외

In [ ]:
# tr_df['대리모 여부'] = tr_df['대리모 여부'].fillna(0.0)
# tt_df['대리모 여부'] = tt_df['대리모 여부'].fillna(0.0)

**`PGD 시술 여부`, `PGS 시술 여부` 결측치 0.0으로 처리 후 사용** >> 제외

In [ ]:
# tr_df['PGD 시술 여부'] = tr_df['PGD 시술 여부'].fillna(0.0)
# tt_df['PGD 시술 여부'] = tt_df['PGD 시술 여부'].fillna(0.0)

# tr_df['PGS 시술 여부'] = tr_df['PGS 시술 여부'].fillna(0.0)
# tt_df['PGS 시술 여부'] = tt_df['PGS 시술 여부'].fillna(0.0)

**레이블 인코딩**

In [ ]:
tr_df, tt_df = pp_base.categorical_to_numerical(tr_df, tt_df, categorical_columns)

**`임신 시도 또는 마지막 임신 경과 연수` 결측치를 -1로 처리**

In [ ]:
tr_df['임신 시도 또는 마지막 임신 경과 연수'] = tr_df['임신 시도 또는 마지막 임신 경과 연수'].fillna(-1)
tt_df['임신 시도 또는 마지막 임신 경과 연수'] = tt_df['임신 시도 또는 마지막 임신 경과 연수'].fillna(-1)

**`총 생성 배아 수`, `미세주입된 난자 수`, `미세주입에서 생성된 배아 수` >> 결측치 0으로 처리, 분포 변환 후 사용**
**`저장된 배아 수`, `미세주입 후 저장된 배아 수`, `해동된 배아 수`, `해동 난자 수`, `수집된 신선 난자 수`, `저장된 신선 난자 수`, `혼합된 난자 수`, `파트너 정자와 혼합된 난자 수`, `기증자 정자와 혼합된 난자 수` >> 결측치 0으로 처리, 분포 변환 후 사용**

In [ ]:
skewed_numerical_columns = [
    '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수',
    '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
    '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
    '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수'
    ]

In [ ]:
tr_df[skewed_numerical_columns] = tr_df[skewed_numerical_columns].fillna(0)
tt_df[skewed_numerical_columns] = tt_df[skewed_numerical_columns].fillna(0)

In [ ]:
tr_df[skewed_numerical_columns] = tr_df[skewed_numerical_columns].apply(np.log1p)
tt_df[skewed_numerical_columns] = tt_df[skewed_numerical_columns].apply(np.log1p)

**`이식된 배아 수` >> 결측치 2로 처리 후 사용** >> 제외

In [ ]:
# tr_df['이식된 배아 수'] = tr_df['이식된 배아 수'].fillna(2)
# tt_df['이식된 배아 수'] = tt_df['이식된 배아 수'].fillna(2)

**`배아 이식 경과일` >> 결측치 5로 처리 후 사용**

In [ ]:
tr_df['배아 이식 경과일'] = tr_df['배아 이식 경과일'].fillna(5)
tt_df['배아 이식 경과일'] = tt_df['배아 이식 경과일'].fillna(5)

**`임신 시도 또는 마지막 임신 경과 연수`를 제외한 나머지 수치형 컬럼들에 대해 결측치 처리**

In [ ]:
# "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", >> 결측치 0으로 처리
# "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수", >> 결측치 0으로 처리
numerical_columns_cp = [
        "이식된 배아 수", "미세주입 배아 이식 수",  "난자 채취 경과일", "난자 해동 경과일",
        "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"
        ]

In [ ]:
tr_df[numerical_columns_cp] = tr_df[numerical_columns_cp].fillna(0)
tt_df[numerical_columns_cp] = tt_df[numerical_columns_cp].fillna(0)

In [ ]:
# split
x_tr, y_tr, x_val, y_val = pp_base.split_train_and_validation(tr_df, 0.2, hparams['seed'], method='custom')

### Training && Validation

In [ ]:
# # Class Weights 구하기
# from sklearn.utils.class_weight import compute_class_weight

# neg_samples = len(tr_df[tr_df['임신 성공 여부'] == 0])
# pos_samples = len(tr_df[tr_df['임신 성공 여부'] == 1])
# class_weights = compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(tr_df['임신 성공 여부']),
#     y=np.array([0] * neg_samples + [1] * pos_samples)
# )

In [ ]:
# CatBoost 모델 정의
catboost_model = CatBoostClassifier(
    iterations=5000,
    learning_rate=3e-2,
    depth=6,
    random_seed=hparams['seed'],
    eval_metric='AUC',
    loss_function='Logloss',
    verbose=100,
    # class_weights=class_weights 
)

In [ ]:
# Training
print("\n🔍 Training CatBoost...")
catboost_model.fit(
    x_tr, y_tr,
    eval_set=(x_val, y_val),
    early_stopping_rounds=50,
    verbose=100,
)

### make submission files

In [ ]:
# pred_proba = catboost_model.predict_proba(tt_df)[:, 1]

In [ ]:
# make_submission_file(pred_proba, dir_name="catboost", model_name="using_all_data", val_roc_auc=None)